In [11]:
from warcio.archiveiterator import ArchiveIterator
import gzip
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment as cmnt
import nltk
from nltk.tokenize import word_tokenize
import re
import urllib
import urllib.request
import os


In [12]:
def get_url_list(target_url): 
    d={}
    proxy="http://commoncrawl.s3.amazonaws.com/"
    stream = requests.get(proxy+target_url, stream=True).raw
    i=0
    for record in ArchiveIterator(stream):
        if record.rec_type == 'response':
                if check_if_covid(record.rec_headers.get_header('WARC-Target-URI'))==True:
                  print("adding")
                  d[i]=(record.rec_headers.get_header('WARC-Target-URI'))
                  i=i+1
    return d

In [13]:

def proper_text(inp):
    if inp.parent.name in ['title', 'meta', '[document]','style', 'script', 'head']:
        return False
    if isinstance(inp, cmnt):
        return False
    return True


def text_from_html(body):
    parsed = BeautifulSoup(body, 'html.parser')
    text = parsed.findAll(text=True)
    beauty_text = filter(proper_text, text)  
    return u" ".join(word.strip() for word in beauty_text)

def get_content(url):
    try:
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 
        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() #
        s=text_from_html(data)
    except:
        s="error"
    return s
        

In [14]:
def check_if_covid(url):
  s=get_content(url)
  s1=word_tokenize(s)
  s2=[]
  for i in s1:
      word=re.sub('[^A-Za-z0-9]+', '', i)
      if len(word)>1:
        if word in ['CORONA','coronavirus','corona','covid','covid-19','covid','Covid','Corona']:
          s2.append(word)  
  if len(s2)>1:
    return True
  else:
    return False

In [ ]:
prefixed = [filename for filename in os.listdir('.') if filename.startswith("2020")]

url={}
url[0]=0
urls={}
for i in range(len(prefixed)):
  with gzip.open(prefixed[i], 'rb') as f:
      for line in f.readlines():
                if url[0]<=1000:
                  op=get_url_list(line.decode()[0:-1])
                  url[0]=url[0]+len(op)
                urls[str(prefixed[i])]=list(op.values())


In [ ]:
print(urls)